In [7]:
import pandas as pd
import numpy as np
from collections import OrderedDict
from sklearn.cross_validation import train_test_split
from six.moves import cPickle
from sklearn.preprocessing import normalize
from main import get_data
from sklearn.decomposition import PCA
from myclasses import NNParameters
#from train_models import perform_zero_prediction, perform_svd, perform_autoencoding, performance_lda, performance_knn
#from plot import cost_function_plot, biplot, roc_plot
from functions import rectifier, sigmoid, linear, cross_entropy, least_squares

In [27]:
from random import uniform
print(uniform(0.7,1.3))

1.2882980047932029


In [28]:
a = (1,2)
b = (*a, 3)
b

(1, 2, 3)

In [64]:

x = np.array([[-1,4,1],[0,10,5],[2,1,2]])
x_norm = (x - np.mean(x, axis=0))
x_norm

array([[-1.33333333, -1.        , -1.66666667],
       [-0.33333333,  5.        ,  2.33333333],
       [ 1.66666667, -4.        , -0.66666667]])

In [65]:
U, D, VT = np.linalg.svd(x_norm)
V_k = VT.T[:,0:2]
z_k = np.dot(x_norm, V_k)
x_k = np.dot(z_k, V_k.T)    
print(z_k, '\n\n', x_k)

[[ 1.33487957  1.94258902]
 [-5.49837434 -0.56871365]
 [ 4.16349477 -1.37387537]] 

 [[-1.33333333 -1.         -1.66666667]
 [-0.33333333  5.          2.33333333]
 [ 1.66666667 -4.         -0.66666667]]


In [68]:
pca = PCA(n_components=2)
z_k = pca.fit_transform(x)
x_k = pca.inverse_transform(z_k)
print(z_k, '\n\n', x_k)

[[ 1.33487957  1.94258902]
 [-5.49837434 -0.56871365]
 [ 4.16349477 -1.37387537]] 

 [[ -1.00000000e+00   4.00000000e+00   1.00000000e+00]
 [  9.43689571e-16   1.00000000e+01   5.00000000e+00]
 [  2.00000000e+00   1.00000000e+00   2.00000000e+00]]


In [113]:
model_name = 'bcer200r100l30r100r200s_0.0073'

# Get the data.
data, regions, ingredients = get_data()

# Separate 1000 recipes for the recipe reconstruction phase
data_reconstruct, data_autoencoder = data[:1000], data[1000:]
_, regions_autoencoder = regions[:1000], regions[:1000]

# Train a new autoencoder network if no network model is given and load⋅
# the results of the autoencoder model, saved as pickle file
if model_name is None:
    model_name  = perform_autoencoding(data_autoencoder, p)
with open('models/' + model_name + '.pkl', 'rb') as f:
    output = cPickle.load(f)
give_neurons, cost_train, cost_test, y_test, y_pred, run_time = output

Read in datafile with 55001 recipies and 381 ingredients.


In [125]:
n_ingredients = 381
rank_list = []
for i in range(1):
    
    recipe = data_reconstruct[0].copy()

    # Get ingredients and remove one randomly
    ingredient_idxs = np.argwhere(recipe != 0).flatten()
    removed_idx = np.random.choice(ingredient_idxs)
    recipe[removed_idx] = 0
    
    # Get output layer of the autoencoder (needs 2D array as input)
    reconstruction = give_neurons(recipe.reshape(1,-1))[-1][0]
    
    # Set ingredients of the recipe on -1  in reconstruction, 
    # we don't want those to in the ranking of our removed ingredient
    reconstruction[recipe == 1] = -1

    # Get the ingredient indexes sorted on reconstruction
    reconstruction_with_idxs = zip(list(reconstruction), list(range(n_ingredients)))
    _, sorted_idxs = zip(*sorted(reconstruction_with_idxs, reverse=True))
    
    # Get index of removed ingredient, add 1 for the rank.
    rank = sorted_idxs.index(removed_idx) + 1
    rank_list.append(rank)
    
    #print(removed_ingredient)
    print(ingredient_idxs)
    
    #print(list(reconstruction))
    #print(list(range(n_ingredients)))
    print(*zip(*sorted(reconstruction_with_idxs)))
    for i, j in reconstruction_with_idxs:
        print(i,j)
    
    break

[ 27  34  75  97 109 227 228 266 363]



In [91]:
for i, j in reconstruction_with_idxs:
    print(i,j)

In [150]:
'{:4.2}'.format(1.123456)

' 1.1'

In [66]:
non_zero_idx=np.argwhere(x!=0).flatten()
remove = np.random.choice(non_zero_idx)
x[remove] = 0
remove

0

In [67]:
x[remove] = 0
x

array([0, 0, 0, 0])

In [16]:
non_zero_idx=np.argwhere(x!=0).flatten()
np.random.shuffle(non_zero_idx)
random_pick=x[non_zero_idx]
random_pick[0]


6.0

In [151]:
print('abc', end ='')
print('\r123')

123


In [6]:
for row in a:
    print(row, 'aaaa')
    print()

[1 2] aaaa

[3 4] aaaa



In [2]:
def get_parameters():
    '''Returns all the parameters of the network and gradient descent 
    as an object of the class NNParameters.'''

    p = OrderedDict()

    #Network settings
    p['has_bias'] = True #True #Add bias to the layers or not
    p['cost_fn'] = least_squares
    p['activation_fn'] = [rectifier, rectifier, linear, rectifier, rectifier, sigmoid]
    p['n_hidden_neurons'] = [200, 50, 2, 50, 200]

    #Gradient descent parameters
    p['test_size'] = 0.02 #Fraction to test the cost
    p['eps_init'] = 0.05 #Half of the init range of the network param
    p['batch_size'] = 10 #8 #Number of observations to use for gradient descent
    p['alpha'] = 0.8 #Inertie coefficient.
    p['delta'] = 0.5 #1 #2 #Learning rate
    p['max_loops'] = 800 #Maximum number of loops over all the data
    p['max_time'] = 18000 #Maximum time
    p['low_cost'] = 0.0 #Lowest cost
    p['cost_update_size'] = 10000 #Number of data to use before updating cost

    return NNParameters(p)

In [136]:
def get_data(remove_NorthAmerican=False, standardize=False):
    """ Import the data, return as numpy array. The ingredients names are⋅
    stored in 'ingredients'.  'Data' contains the ingredients for each recipe,
    coded as 0/1. 'Regions' contains the region of origin of the recipe. """
    
    # Read in the data
    df = pd.read_csv('datasets/ReceptenBinair_minimun2ingredients.csv', sep=';')
    values = df.values
    ingredients = np.array(df.columns)[1:]
        
    # Remove NorthAmerican recipe
    if remove_NorthAmerican:
        values = values[values[:,0] != 'NorthAmerican']                                                                    
    
    # Shuffle the data with seed
    np.random.seed(42)
    np.random.shuffle(values)
    
    # Split into regions and the data containing ingredients coded as 0/1
    regions = values[:,0]
    data = values[:,1:].astype(int)
    
    # Standardize (In combination with remove NorthAmerican can create nan values!)
    if standardize:
        data = (data - np.mean(data, axis=0)) / np.std(data, axis=0)
    
    # Print dimensions and return regions, data and ingredients
    out_text = 'Read in datafile with {} recipies and {} ingredients.'
    print(out_text.format(len(data), len(ingredients)))
    return regions, data, ingredients

In [139]:
a,b,c = get_data(standardize=True)

Read in datafile with 55001 recipies and 381 ingredients.


In [140]:
b

array([[-0.20848079, -0.00426401, -0.06351629, ..., -0.25572382,
        -0.13848734, -0.14239131],
       [-0.20848079, -0.00426401, -0.06351629, ..., -0.25572382,
        -0.13848734, -0.14239131],
       [-0.20848079, -0.00426401, -0.06351629, ...,  3.91046869,
        -0.13848734, -0.14239131],
       ..., 
       [-0.20848079, -0.00426401, -0.06351629, ..., -0.25572382,
        -0.13848734, -0.14239131],
       [-0.20848079, -0.00426401, -0.06351629, ..., -0.25572382,
        -0.13848734, -0.14239131],
       [-0.20848079, -0.00426401, -0.06351629, ...,  3.91046869,
        -0.13848734, -0.14239131]])

In [124]:
data = (data - np.mean(data, axis=0)) / np.std(data, axis=0)


In [135]:
np.std(df, axis=0)

almond                     0.199797
angelica                   0.004264
anise                      0.063261
anise_seed                 0.039050
apple                      0.203264
apple_brandy               0.025218
armagnac                   0.105235
apricot                    0.014141
artemisia                  0.015372
artichoke                  0.083694
asparagus                  0.088782
avocado                    0.107737
bacon                      0.193247
baked_potato               0.012791
balm                       0.007385
banana                     0.131824
barley                     0.064810
bartlett_pear              0.020445
basil                      0.252895
bay                        0.161337
bean                       0.182253
beech                      0.004264
beef                       0.282338
beef_broth                 0.122202
beef_liver                 0.013483
beer                       0.072421
beet                       0.064391
bell_pepper                0

In [133]:
df = pd.read_csv('datasets/ReceptenBinair_minimun2ingredients.csv', sep=';')
    
#Remove empty ingredients
#df.sum(axis=0)

In [134]:
#np.std(data, axis=0)

In [ ]:
df.sum(axis=0)

In [7]:
data_train, data_test = train_test_split(data_filtered, test_size=0.3, random_state=42)
regions_train, regions_test = train_test_split(regions_filtered, test_size=0.3, random_state=42)

k = 15

#data_train = data_train[:20000]
#regions_train = regions_train[:20000]
k_vals = [1,5,10,12,15,18,20,30,40]
for k in k_vals:
    print(k)
    performance_knn(data_train, data_test, regions_train, regions_test, 'raw_knn', k)


1
knn accuracy: 0.6426
5
knn accuracy: 0.6593
10
knn accuracy: 0.6714
12
knn accuracy: 0.6691
15
knn accuracy: 0.6734
18
knn accuracy: 0.6759
20
knn accuracy: 0.6730
30
knn accuracy: 0.6741
40
knn accuracy: 0.6734


In [5]:
'''
for k in range(1, 21, 10):
    performance_knn(data_train, data_test, regions_train, regions_test,
    'raw_knn', k)'''

Read in datafile with 55001 recipies and 381 ingredients.


"\nfor k in range(1, 21, 10):\n    performance_knn(data_train, data_test, regions_train, regions_test,\n    'raw_knn', k)"

In [4]:
get_data()

In [20]:
num = 32555
for num in range(10001, 30000, 100):
    for el, name in zip(list(df.iloc[num]), list(df.columns)):
        if el:
            print(name)
    country = df.iloc[num].name
    print(country)
    if country != 'NorthAmerican':
        input('')

NameError: name 'df' is not defined

In [4]:
#Auto encoder
model_name  = perform_autoencoding(data, p) 
with open('models/' + model_name + '.pkl', 'rb') as f:
    give_neurons, cost_train, cost_test, y_test, y_pred = cPickle.load(f)
    
#Get the bottleneck neurons of all data
all_neurons = give_neurons(data.tolist())
bottleneck_neurons = all_neurons[p.bottleneck_index]

NameError: name 'data' is not defined

In [5]:
from matplotlib import pyplot as plt
#SVD
p = get_parameters()
data, regions, ingredients = get_data(centre=True, normalize=True)
cost_svd, data_svd = perform_svd(data, p)

#Make biplot of the bottleneck neurons and SVD reduction
biplot(data_svd, regions, 'svd_{:0.4f}'.format(cost_svd))

Read in datafile with 55001 recipies and 381 ingredients.
Performing singular value decomposition...


In [10]:
        biplot(bottleneck_neurons, regions, model_name)


    #Filter out NorthAmerican for region prediction. NorthAmerican is 
    #contains recipes of all regions and dominates in numbers, 
    #which makes the prediction trivial
    features_filtered = []
    regions_filtered  = []
    for features, region in zip(bottleneck_neurons, regions):
        if region != 'NorthAmerican':
            features_filtered.append(features)
            regions_filtered.append(region)

    biplot(features_filtered, regions_filtered, model_name + '_filter')

    #Measure performance without NorthAmerican. Different train/test split 
    #than autoencoder training since the outcome variable 'region' wasn't used.
    data_train, data_test = train_test_split(features_filtered, test_size=0.3,
            random_state=42)
    regions_train, regions_test = train_test_split(regions_filtered, 
            test_size=0.3, random_state=42)

    #lda
    regions_predicted = performance_lda(data_train, data_test, regions_train, 
            regions_test)
    biplot(data_test, regions_predicted, 'lda_' + model_name)

    #knn
    regions_predicted = performance_knn(data_train, data_test, regions_train, 
            regions_test, model_name)
    biplot(data_test, regions_predicted, 'knn_' + model_name)

    input('Press any key to close figures')

Read in datafile with 55001 recipies and 381 ingredients.


In [3]:
import numpy as np
def lin(x):
    return x
def lin_grad(x):
    y = x.copy()
    y.fill(1)
    return y
def rec(x):
    y = x.copy()
    y[y<0] = 0
    return y
def rec_grad(x):
    return rec(x)/x
def sigm(x):
    return 1/(1 + np.exp(-x))
def sigm_grad(x):
    return sigm(x) * (1-sigm(x))

In [5]:
import numpy as np
import matplotlib.pyplot as plt

x = np.arange(-9.999999999,10,0.1)

#box = dict(facecolor='yellow', pad=5, alpha=0.2)

fig = plt.figure()
fig.subplots_adjust(left=0.2, wspace=0.6)

funcs = [lin, lin_grad, sigm, sigm_grad, rec, rec_grad]
yrange = [(-10,10),(-0.2,1.2),(-0.2,1.2),(-0.05,0.3),(-2,10),(-0.2,1.2)]

for i, func in enumerate(funcs):
    ax = fig.add_subplot(int('32' + str(i+1)))
    ax.axhline(y=0, color='k')
    ax.axvline(x=0, color='k')
    ax.plot(x, func(x))
    ax.set_ylim(*yrange[i])
    if i < 4:
        ax.set_xticklabels([])
    if i == 0:
        ax.set_title('Function')
        ax.set_ylabel('Linear')
    if i == 1:
        ax.set_title('Function derivative')
    if i == 2:
        ax.set_ylabel('Sigmoid')
    if i == 4:
        ax.set_ylabel('Rectifier')

plt.savefig('act_fns.eps', format='eps', dpi=1000)    

In [11]:
import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

x = np.array([50,30,20,10,5,4,3,3,3,2,2,2,1,1,1])*0.5

# the histogram of the data
plt.hist(x, facecolor='green', bins=15,normed=1)#50, normed=1, facecolor='green', alpha=0.75)

plt.xlabel('Smarts')
plt.ylabel('Probability')
#plt.axis([40, 160, 0, 0.03])
plt.grid(True)

plt.show()